# O Gradiente Descendente

O Gradiente Descendente é um algoritmo de otimização usado para minimizar alguma função objetivo iterativamente. É popularmente usado no aprendizado de máquina para otimizar modelos complexos e não-lineares, como redes neurais.

Para ilustrar os principais pontos do algoritmo, vamos implementá-lo sem o auxílio do `sklearn`.

**EXEMPLO**

Vamos imaginar que você esteja tentando prever o tempo de sentença de um réu a partir de um dataset de condenações anteriores de vários juízes.

Condenações anteriores ($X$) | Sentenças (em meses) ($Y$)
:---------------------------:|---------------------------
0 | 12
3 | 13
1 | 15
0 | 19
6 | 26
5 | 27
3 | 29
4 | 31
10 | 40
8 | 48

## O Algoritmo do Zero

**Bibliotecas**

In [52]:
import numpy as np
import plotly.express as px
import pandas as pd

**Ingestão**

In [45]:
# Dados
X = np.array([0, 3, 1, 0, 6, 5, 3, 4, 10, 8])
Y = np.array([12, 13, 15, 19, 26, 27, 29, 31, 40, 48])

**Preparação**

Vamos adiciona uma coluna de 1's ao X para que possamos tratar o intercepto como um parâmetro a ser aprendido, exatamente como os coeficientes das outras variáveis independentes. Isso dá mais flexibilidade ao modelo.

In [63]:
m = len(X)

# Adicionando uma coluna de uns para o termo de bias (intercept)
X_b = np.c_[np.ones((m, 1)), X]

**Implementação**

*FUNÇÃO DE CUSTO*

Vamos começar definindo uma função de custo. No caso desse cadernos, vamos utilizar o Erro Quadrático Médio.

$$
EQM = \frac{1}{m} \sum_{i=1}^{m}(y_{i} - \hat{y}_{i})^2
$$

In [35]:
def EQM(Y, Y_pred):
    m = len(Y)
    custo = (1/(2*m)) * np.sum((Y - Y_pred)**2)
    return custo

> No contexto de aprendizado de máquina, "custo" e "perda" são frequentemente usados como sinônimos, mas tecnicamente eles têm diferenças sutis.

>> **Função de Perda (Loss Function)**: A função de perda é uma medida do erro para uma única amostra de treinamento. Ela calcula a diferença entre a previsão do modelo para uma determinada amostra e o valor verdadeiro dessa amostra. Um exemplo comum de função de perda é o erro quadrático médio para problemas de regressão.

>> **Função de Custo (Cost Function)**: A função de custo, por outro lado, é uma medida do erro do modelo sobre todo o conjunto de treinamento (ou seja, é a média das funções de perda sobre todo o conjunto de treinamento). O objetivo durante o treinamento de um modelo de aprendizado de máquina é minimizar a função de custo.

*GRADIENTE DESCENDENTE*

O algoritmo calcula o gradiente da função de custo em relação a esses parâmetros. O gradiente é um vetor que aponta na direção de maior aumento da função, e seu tamanho é proporcional à taxa de aumento. A ideia-chave por trás do algoritmo é que, se você quer minimizar uma função, uma maneira de fazer isso é mover na direção oposta àquela que aumenta a função mais rapidamente - daí o nome "gradiente descendente".

Para encontrar o gradiente, precisamos calcular as derivadas parciais de cada um dos atributos. No nosso caso, `theta_0` e `theta_1`.

*Derivada parcial com relação à `theta_0`*

In [36]:
# Esta função calcula a derivada parcial da função de custo em relação ao parâmetro theta_0, que é o viés ou intercepto da linha de regressão.
def derivada_parcial_theta_0(X, Y, theta_0, theta_1):
    m = len(Y)
    soma = 0
    for i in range(m):
        soma += theta_0 + theta_1*X[i] - Y[i]
    return (1/m) * soma

Esta é a representação matemática dessa implementação.

$$
D_{\theta_{0}} = \frac{-1}{m} \sum_{i=1}^{m}(y_{i} - \hat{y_{i}})
$$

*Derivada parcial com relação à `theta_1`*

In [37]:
# Esta função é muito semelhante à anterior, mas calcula a derivada parcial da função de custo em relação ao parâmetro theta_1, que é a inclinação da linha de regressão.
def derivada_parcial_theta_1(X, Y, theta_0, theta_1):
    m = len(Y)
    soma = 0
    for i in range(m):
        soma += (theta_0 + theta_1*X[i] - Y[i]) * X[i]
    return (1/m) * soma

Esta é a representação matemática dessa implementação.

$$
D_{\theta_{1}} = \frac{-1}{m} \sum_{i=1}^{m}x_{i}(y_{i} - \hat{y_{i}})
$$

*A função de previsão*

In [38]:
def predict(X, theta_0, theta_1):
    return theta_0 + theta_1 * X

In [64]:
def gradiente_descendente(X, Y, theta_0, theta_1, alpha, epochs):
    m = len(Y)
    custos = []
    theta_0s = []
    theta_1s = []

    for i in range(epochs):
        # O vetor gradiente será composto por todas as derivadas
        grad_0 = derivada_parcial_theta_0(X, Y, theta_0, theta_1)
        grad_1 = derivada_parcial_theta_1(X, Y, theta_0, theta_1)

        # Calcula os novos coeficientes 
        theta_0 = theta_0 - alpha * grad_0
        theta_1 = theta_1 - alpha * grad_1

        # Faz as previsões com os novos coeficientes
        Y_pred = predict(X, theta_0, theta_1)

        # Armazena o custo e os coeficientes utilizados
        custos.append(EQM(Y, Y_pred))
        theta_0s.append(theta_0)
        theta_1s.append(theta_1)
        
    return [theta_0s, theta_1s], custos


*Rodando o algoritmo*

Para executar o gradiente descendente, temos que definir os hiperparâmetros.

In [50]:
# Inicializando os parâmetros
theta_0 = 0 # Um valor aleatório
theta_1 = 0 # Um valor aleatório

# Taxa de aprendizado
alpha = 0.01

# Número de iterações
n_epochs = 1000

In [68]:
# Rodar o algoritmo do gradiente descendente
thetas, custos = gradiente_descendente(X, Y, theta_0, theta_1, alpha, n_epochs)

thetas0 = thetas[0]
thetas1 = thetas[1]

**Visualizando os resultados**

In [69]:
# Converter os resultados para um DataFrame para visualização
resultado = pd.DataFrame({'Theta0': [theta for theta in thetas0], 'Theta1': [theta for theta in thetas1], 'Custo': custos})

In [67]:

# Gráfico do custo em relação a Theta0
fig1 = px.scatter(resultado, x='Theta0', y='Custo', title='Custo vs Theta0')
fig1.show()

In [71]:
# Gráfico do custo em relação a Theta0
fig2 = px.scatter(resultado, x='Theta1', y='Custo', title='Custo vs Theta1')
fig2.show()

In [72]:
fig = px.scatter_3d(resultado, x='Theta0', y='Theta1', z='Custo', title='Visualização do Gradiente Descendente')
fig.update_traces(marker=dict(size=4))
fig.show()